In [1]:
from bs4 import BeautifulSoup
import requests
import os
import shutil

MUSCLE_LIST = ["neck", "trapezius", "shoulders", "chest", "back", "erector-spinae", "biceps", "triceps", "forearm", "abs", "leg", "calf", "hip", "cardio", "full-body"]

for directory in MUSCLE_LIST:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [2]:
#SCRAPING FUNCTIONS

def add_page(link):
    NUMBERS = [str(i) for i in range(1,100)]
    link_parts = link.split('/')
    if link_parts[-1] in NUMBERS:
        val = int(link_parts.pop())
        link_parts.append(NUMBERS[val])
    else :
        link_parts.extend(['page', '2'])
    new_link = "/".join(link_parts)
    return new_link

def download_images(images_link, muscle):
    for image_url in images_link:
        image_url_parts = image_url.split("/")
        response = requests.get(image_url, stream=True)
        with open(muscle + "/" + image_url_parts[-1], "wb") as img:
            shutil.copyfileobj(response.raw, img)

def list_all_exercises_gif(path):
    gif_files = []
    for _, _, files in os.walk(path):
        for file in files:
            if file.endswith(".gif"):
                gif_files.append(file[:-4])
    return gif_files

In [3]:
for muscle in MUSCLE_LIST:
    link = "https://fitnessprogramer.com/exercise-primary-muscle/" + muscle
    img_count = 0
    while True:
        try : 
            response = requests.get(link)
            if response.status_code != 200:
                raise Exception("Link error")
            page_content = response.content
            soup = BeautifulSoup(page_content, 'lxml')
            images = soup.find_all("img")
            images_links = [image["src"] for image in images if image["src"].endswith("gif")]
            download_images(images_links, muscle)
            filepath = muscle + '/' + muscle + "_linkslist.txt"
            with open(filepath, "a+") as file : 
                file.write("\n".join(images_links))
                file.close()
            link = add_page(link)
            img_count += len(images_links)
        except:
            print(f"Saved {img_count} GIF files for {muscle}.")
            break

Saved 23 GIF files for neck.
Saved 53 GIF files for trapezius.
Saved 192 GIF files for shoulders.
Saved 153 GIF files for chest.
Saved 125 GIF files for back.
Saved 56 GIF files for erector-spinae.
Saved 69 GIF files for biceps.
Saved 104 GIF files for triceps.
Saved 30 GIF files for forearm.
Saved 154 GIF files for abs.
Saved 233 GIF files for leg.
Saved 40 GIF files for calf.


In [ ]:
all_exercises = list(set(list_all_exercises_gif(os.getcwd())))
all_links = list_all_exercises_gif(os.getcwd())

print(f'No duplicates') if len(all_links) == len(all_exercises) else print(f"Links : {len(all_links)}, Unique exercises : {len(all_exercises)}")

with open("exercises_list.txt", "w+") as f :
    f.write("\n".join(all_exercises))
    f.close()